In [42]:
import os
import sys
import numpy as np
sys.path.append('../../')
import pandas as pd
import numpy as np
from loguru import logger

from utils.token_price import get_token_price


def portfolio_token_analysis(balance):
    df = pd.DataFrame(balance, columns = ['token','address','amount'])
    df = df.groupby('token')['amount'].sum().reset_index()
    for index, row in df.iterrows():
        token_price = get_token_price(row['token']) if row['token'].lower() != 'usd' else 1
        df.at[index, 'price'] = float(token_price)
        df.at[index, 'value'] = float(token_price) * float(row['amount'])
    total_value = df['value'].sum()
    logger.info(f'total_value is {total_value}')
    df['pct'] = df['value'] / total_value
    df = df.sort_values('pct',ascending=False)
    return df.reset_index().drop('index', axis=1)

df = portfolio_token_analysis(balance)
# df = portfolio_token_analysis(tp_balance)
df

2024-03-22 17:51:12.346 | INFO     | utils.token_price:get_token_price:51 - return binance price - arb:1.65540000
2024-03-22 17:51:12.623 | INFO     | utils.token_price:get_token_price:51 - return binance price - atom:11.68000000
2024-03-22 17:51:12.933 | INFO     | utils.token_price:get_token_price:51 - return binance price - blur:0.57390000
2024-03-22 17:51:13.159 | INFO     | utils.token_price:get_token_price:51 - return binance price - bnb:582.70000000
2024-03-22 17:51:13.371 | INFO     | utils.token_price:get_token_price:51 - return binance price - btc:65457.55000000
2024-03-22 17:51:13.596 | INFO     | utils.token_price:get_token_price:51 - return binance price - cfx:0.42180000
2024-03-22 17:51:13.825 | INFO     | utils.token_price:get_token_price:51 - return binance price - eth:3460.00000000
2024-03-22 17:51:14.247 | INFO     | utils.token_price:get_token_price:59 - return okx price - gpt:0.49403
2024-03-22 17:51:14.481 | INFO     | utils.token_price:get_token_price:51 - return 

,token,amount,price,value,pct
0,eth,7.40,3460.00000,25604.0000,0.296745
1,sui,8630.00,1.77840,15347.5920,0.177875
2,bnb,16.96,582.70000,9882.5920,0.114537
3,okb,100.00,60.57000,6057.0000,0.070199
4,btc,0.09,65457.55000,5891.1795,0.068277
5,pyth,4001.00,0.92230,3690.1223,0.042768
6,usd,2500.00,1.00000,2500.0000,0.028974
7,cfx,5724.00,0.42180,2414.3832,0.027982
8,link,132.00,18.20900,2403.5880,0.027857
9,manta,595.00,3.08900,1837.9550,0.021301


In [ ]:
def get_address_group(address):
    if address.split('-')[0] == 'onchain':
        return address.split('-')[0] + '-' + address.split('-')[1]
    else:
        return address

def portfolio_address_analysis(balance):
    df = pd.DataFrame(balance, columns = ['token','address','amount'])
    for index, row in df.iterrows():
        token_price = get_token_price(row['token']) if row['token'].lower() != 'usd' else 1
        df.at[index, 'price'] = float(token_price)
        df.at[index, 'value'] = float(token_price) * float(row['amount'])
    df['address_group'] = df['address'].apply(lambda x: get_address_group(x))
    df = df.groupby('address_group')['value'].sum().reset_index()
    total_value = df['value'].sum()
    logger.info(f'total_value is {total_value}')
    df['pct'] = df['value'] / total_value
    df = df.sort_values('pct',ascending=False)
    return df

df = portfolio_address_analysis(balance)
df

In [38]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd
import mplfinance as mpf

from utils.token_price import get_binance_historical_data
from utils.token_price import get_okx_historical_data


def get_btc_based_trend(token,interval='1h'):
    token_df = get_binance_historical_data(token,interval)
    btc_df = get_binance_historical_data('btc',interval)
    
    for col in ['open','high','low','close','volume']:
        token_df[col] = token_df[col].astype(float)
        btc_df[col] = btc_df[col].astype(float)
    
    token_degree = 0
    while np.floor(token_df['open']*pow(10,token_degree)).max() == 0:
        token_degree += 1
    
    df = pd.merge(token_df,btc_df,on='timestamp',how='inner')
    for col in ['open','high','low','close']:
        df[col] = df[col+'_x']/df[col+'_y']*pow(10,token_degree+5)
    df.set_index('timestamp', inplace=True)
    df = df.loc[:,['open','high','low','close']]
    
    return df

token = 'cfx'
df = get_btc_based_trend(token,interval='1d')
mpf.plot(df, type='candle', style='charles', title='Candlestick Chart', ylabel='Price', figscale=1.5, mav=(5,30))

ImportError: cannot import name 'get_binance_historical_data' from 'utils.token_price' (/Users/yuezhang/Documents/dex/utils/token_price.py)